# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','RevTrdLearning.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

Plot group learning curve

In [ ]:
def plot_learning_curve(root, ax, profile, TaskParamToPlot, animalList=None, NSession=30, plotAnimal=False):
    assert isinstance(TaskParamToPlot,str)
    #learining curve
    if animalList is None:
        animalList= batch_get_animal_list(root,profile)
    if plotAnimal:
        colors=get_colors(len(animalList),colormap='cool')
    
    data=np.ones((NSession,len(animalList)))*np.nan
    x=np.arange(1,NSession+1)
    
    badAnimals=[]
    for i,animal in enumerate(animalList):
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot, profile=profile)
        if len(out) < NSession:
            badAnimals.append(i)
            continue
        data[:,i]=out[:NSession]
        if plotAnimal:
            ax.plot(x, out[:NSession],color=colors[i],lw=.5,alpha=.5,zorder=0)
    
    data=np.delete(data,badAnimals,axis=1)
    
    y=np.nanmedian(data,axis=1)
    Gline=ax.plot(x, y,color='xkcd:black',zorder=4)
    ax.fill_between(x,y1=np.nanpercentile(data,25,axis=1),y2=np.nanpercentile(data,75,axis=1),
                    fc='xkcd:black',ec=None,alpha=.4,zorder=3)

    ax.set_xlim([x[0]-1,x[-1]+1])
    xtick=[1]
    for i in range(1,NSession+1):
        if i%10==0:
            xtick.append(i)
    ax.set_xticks(xtick)
    ax.spines['bottom'].set_bounds(x[0],x[-1])
    ax.set_xlabel('Learning session #')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
#     ax.set_ylim([0,10])
#     ax.set_yticks([0,7,10])
#     ax.set_ylabel(TaskParamToPlot)
    
    return data

In [ ]:
if "__file__" not in dir():

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control']
             }

    TaskParamToPlot='median entrance time (sec)'
    NSession =30
    
    plt.close('all')
    fig=plt.figure(figsize=(10,5))
    ax=fig.add_subplot(111);
    
    plot_learning_curve(root, ax, profile, TaskParamToPlot, NSession=NSession)
    ax.set_ylabel('Entrance Times (s)')
    plt.show()

---

AntiSequential trials: Starting from Back for the Reverse Treadmill

In [ ]:
def _anti_sequential_trials(data, xFront=60):
    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]<xFront and pos[0]<xFront:
            continue

        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)

    return len(seqTrial)/data.nTrial

In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    print(_anti_sequential_trials(data))

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'
       

    
    #===============================================
    
    # GRID 1 PARAMS
    
    profilecontrol1={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']}
    
    profilePre1={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['10','8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    animalList1=['Rat372','Rat337','Rat341','Rat366','Rat365','Rat333','Rat329','Rat325','Rat345']
    
    TaskParamToPlot1='median entrance time (sec)'
    NSession1 =30
    
    TaskParamToPlot2="standard deviation of entrance time"
    TaskParamToPlot3 = _anti_sequential_trials.__name__



Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(5,1.1)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    ##########################################
    # 1: Lesion Size Loc LATE
    gs1= fig.add_gridspec(nrows=1, ncols=3, left=0, bottom=0, right=1, top=1,wspace=.25)
    ax1= fig.add_subplot(gs1[0])
    ax2= fig.add_subplot(gs1[1])
    ax3= fig.add_subplot(gs1[2])
    
    data1=plot_learning_curve(root, ax1, profilePre1, TaskParamToPlot1, NSession=NSession1,)
    data2=plot_learning_curve(root, ax2, profilePre1, TaskParamToPlot2, NSession=NSession1,)
    data3=plot_learning_curve(root, ax3, profilePre1, TaskParamToPlot3, NSession=NSession1,)
    
    ax1.text(NSession1,2,f'$n={data1.shape[1]}$ rats', ha='right',va='bottom',fontsize='xx-small')
    
    ax1.set_ylim([2,8])
    ax1.set_ylabel('Entrance time (s)')
    ax2.set_ylim([2,5])
    ax2.set_ylabel('$SD_{ET}$ (s)')
    ax3.set_ylim([.2,.7])
    ax3.set_yticks(np.arange(.2,.71,.1))
    ax3.set_yticklabels([20,30,40,50,60,70])
    ax3.set_ylabel('%Anti Routine')


    
    
#     fig.align_ylabels([axes1[0],axes1[1],axes1[2]])
    ############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(ax1,ax2,)
#     OFFX=np.array([.06]*len(AXES))
#     OFFY=np.array([.03]*len(AXES))
#     OFFX[[0,2]]=0.05
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','RevTrdLearning.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()